# Expermenting with Conditional Random Field Model
-------------

Loading data and models for CRF cross validations

In [1]:
import __init__
import models.crf as crf
import models.cross_validator as cross_validator
import models.datahandler as datahandler
from models.datamodel import DataModel
import models.outputmaker as outputmaker
import evaluator.metrics as metrics

trial = datahandler.load_train('../data/dataset/tsd_trial.csv', verbose=True)
train = datahandler.load_train('../data/dataset/tsd_train.csv', verbose=True)

/home/hb/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


Initialize `CRF Model` Methods

In [2]:
model = crf.CRF()

datamodel = DataModel(model='crf')

output_maker = outputmaker.crf_output

evaluator = metrics

### Cross Validations

In [3]:
print("TRIAL DATA:")
data = trial
cv=5
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

cv=10
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

0it [00:00, ?it/s]

TRIAL DATA:


5it [01:06, 13.35s/it]
0it [00:00, ?it/s]

---------------------------------------------------------------------------
5-Fold Cross Validation Averaged Results:
F1 = 0.40 ± 0.03, 	 P = 0.45 ± 0.03, 	 R = 0.40 ± 0.03
---------------------------------------------------------------------------


10it [02:32, 15.26s/it]

---------------------------------------------------------------------------
10-Fold Cross Validation Averaged Results:
F1 = 0.41 ± 0.02, 	 P = 0.47 ± 0.02, 	 R = 0.41 ± 0.02
---------------------------------------------------------------------------


In [4]:
print("TRAIN DATA:")
data = train
cv=5
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

cv=10
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

0it [00:00, ?it/s]

TRAIN DATA:


5it [23:54, 286.86s/it]
0it [00:00, ?it/s]

---------------------------------------------------------------------------
5-Fold Cross Validation Averaged Results:
F1 = 0.50 ± 0.00, 	 P = 0.58 ± 0.01, 	 R = 0.50 ± 0.00
---------------------------------------------------------------------------


10it [54:44, 328.44s/it]

---------------------------------------------------------------------------
10-Fold Cross Validation Averaged Results:
F1 = 0.51 ± 0.00, 	 P = 0.58 ± 0.00, 	 R = 0.50 ± 0.00
---------------------------------------------------------------------------


### Train on train data, test on trial data

In [4]:
model = crf.CRF()

datamodel = DataModel(model='crf')

output_maker = outputmaker.crf_output

evaluator = metrics

In [5]:
X_train, y_train , _, _ = cross_validator.load_data(train, [i for i in range(train.shape[0])], 
                                                    datamodel, logger=True)

X_test, y_test , _, test_texts = cross_validator.load_data(trial, [i for i in range(trial.shape[0])], 
                                                           datamodel, logger=True)

100%|██████████| 690/690 [00:11<00:00, 61.67it/s]


In [6]:
model.fit(X_train, y_train)

preds = model.predict(X_test)

predictions = output_maker(X_test, test_texts, preds)

f1, p, r = evaluator.evaluate(gold = y_test, predictions = predictions)

print("F1:{}, \t P:{}, \t R:{}".format(f1, p, r))

F1:0.5330572248549511, 	 P:0.6112130602357639, 	 R:0.5232665855932735


In [7]:
crf = model.get_model()

In [9]:
import eli5

eli5.explain_weights(crf, top=50)

Explanation(estimator='CRF(keep_tempfiles=None, max_iterations=200)', description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='normal', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='-1:word.lower():foolish', weight=2.255805, std=None, value=None), FeatureWeight(feature='-1:word.lower():bullshit', weight=1.627119, std=None, value=None)], neg=[FeatureWeight(feature='word.lower():stupidity', weight=-2.867514, std=None, value=None), FeatureWeight(feature='word.lower():morons', weight=-2.769092, std=None, value=None), FeatureWeight(feature='word.lower():idiots', weight=-2.669159, std=None, value=None), FeatureWeight(feature='word.lower():hypocrite', weight=-2.488741, std=None, value=None), FeatureWeight(feature='word.lower():idiotic', weight=-2.343871, std=None, value=None), FeatureWeight(feature='word.lower():stupidest', weight=-2.290776, std=None, value=None), FeatureWeight(feature='word.lower():idiot', weight=-2.261538, std=None, value=None), FeatureWeight(feature='word.lower():damned', weight=-2.237741, std=None, value=None), FeatureWeight(feature='word.lower():fucking', weight=-2.220493, std=None, value=None), FeatureWeight(feature='word.lower():imbecile', weight=-2.153985, std=None, value=None), FeatureWeight(feature='word.lower():ridiculous', weight=-2.120283, std=None, value=None), FeatureWeight(feature='word.lower():sucks', weight=-2.109719, std=None, value=None), FeatureWeight(feature='word.lower():fool', weight=-2.10725, std=None, value=None), FeatureWeight(feature='word.lower():hypocrites', weight=-2.07628, std=None, value=None), FeatureWeight(feature='word.lower():garbage', weight=-2.054797, std=None, value=None), FeatureWeight(feature='word.lower():ignorant', weight=-2.04778, std=None, value=None), FeatureWeight(feature='word.lower():dumber', weight=-2.009785, std=None, value=None), FeatureWeight(feature='word.lower():silly', weight=-1.989646, std=None, value=None), FeatureWeight(feature='word.lower():losers', weight=-1.974771, std=None, value=None), FeatureWeight(feature='word.lower():clown', weight=-1.95224, std=None, value=None), FeatureWeight(feature='word.lower():suckers', weight=-1.910634, std=None, value=None), FeatureWeight(feature='word.lower():stupid', weight=-1.900802, std=None, value=None), FeatureWeight(feature='word.lower():dumbest', weight=-1.888488, std=None, value=None), FeatureWeight(feature='word.lower():trash', weight=-1.838025, std=None, value=None), FeatureWeight(feature='word.lower():moron', weight=-1.835646, std=None, value=None), FeatureWeight(feature='word.lower():assholes', weight=-1.826841, std=None, value=None), FeatureWeight(feature='word.lower():lunatics', weight=-1.813138, std=None, value=None), FeatureWeight(feature='word.lower():hypocritical', weight=-1.790717, std=None, value=None), FeatureWeight(feature='word.lower():kill', weight=-1.788597, std=None, value=None), FeatureWeight(feature='word.lower():coward', weight=-1.762596, std=None, value=None), FeatureWeight(feature='word.lower():bastards', weight=-1.741136, std=None, value=None), FeatureWeight(feature='word.lower():imbeciles', weight=-1.7359, std=None, value=None), FeatureWeight(feature='word.lower():darn', weight=-1.707027, std=None, value=None), FeatureWeight(feature='word.lower():asshole', weight=-1.702082, std=None, value=None), FeatureWeight(feature='word.lower():dick', weight=-1.652091, std=None, value=None), FeatureWeight(feature='word.lower():pathetic', weight=-1.642506, std=None, value=None), FeatureWeight(feature='word.lower():whore', weight=-1.627688, std=None, value=None), FeatureWeight(feature='word.lower():damn', weight=-1.622251, std=None, value=None), FeatureWeight(feature='word.lower():fools', weight=-1.617206, std=None, value=None), FeatureWeight(feature='word.lower():bitch', weight=-1.613892, std=None, value=None), FeatureWeight(feature='word.lower():moronic', weight=-1.608864, std=None, value=None), FeatureWeight(feature='word.lower():sucking', weight=-